In [45]:
#r “nuget:CsvHelper” 

Installing package CsvHelper.done!

Successfully added reference to package CsvHelper, version 15.0.4

In [46]:
using System;
using System.Globalization;
using System.IO;
using CsvHelper.Configuration;
using CsvHelper;
using System.Linq;
using System.Collections.Generic;

In [47]:
DateTime  startDate =  new DateTime(2019, 01, 01);;
const string directory = "\\Education\\property_analysis\\data\\";
const string propertyFile = "ppd_data_Ilford";
const string suffix = "csv";

string filePath = Path.Combine(directory, propertyFile + "." + suffix);
filePath

\Education\property_analysis\data\ppd_data_Ilford.csv

In [48]:
List<string> codes = new List<string>() {"IG1", "IG2", "IG3", "IG4", "IG5", "IG6" };
codes

index,value
0,IG1
1,IG2
2,IG3
3,IG4
4,IG5
5,IG6


In [49]:
Dictionary<string, string> propertyTypeLookup = new Dictionary<string, string>()
{{"D","Detached"},{"S","Semi-detached"},{"T", "Terraced"},{"F", "Flat"}};

propertyTypeLookup

key,value
D,Detached
S,Semi-detached
T,Terraced
F,Flat


In [50]:
//error handle the file

In [51]:
public class LandRegistryOpenData
{
    public double Price { get; set; }
    public DateTime DateOfTransfer { get; set; }
    public string PostCode { get; set; }
    public string PropertyType { get; set; }
    public string Street { get; set; }
    public string City { get; set; }
    public string District { get; set; }
    public string County { get; set; }

}

public class LandRegistryOpenDataMap : ClassMap<LandRegistryOpenData>
{
    public LandRegistryOpenDataMap()
    {

        Map(m => m.Price).Index(1);
        Map(m => m.DateOfTransfer).Index(2);
        Map(m => m.PostCode).Index(3);
        Map(m => m.PropertyType).Index(4);
        Map(m => m.Street).Index(9);

        Map(m => m.City).Index(11);
        Map(m => m.District).Index(12);
        Map(m => m.County).Index(13);
    }

}

In [52]:
static string GetPostcode(string value) => value.Split(" ")[0]; 


static string GetPropertyType(string value, Dictionary<string,string> property) 
    => property.ContainsKey(value) ? property[value] : value;


public static double Median(this IEnumerable<double> source)
{
        if (source.Count() == 0)
        {
            throw new InvalidOperationException("Cannot compute median for an empty set.");
        }

        var sortedList = from number in source
                         orderby number
                         select number;

        int itemIndex = (int)sortedList.Count() / 2;

        if (sortedList.Count() % 2 == 0)
        {
            // Even number of items.
            return (sortedList.ElementAt(itemIndex) + sortedList.ElementAt(itemIndex - 1)) / 2;
        }
        else
        {
            // Odd number of items.
            return sortedList.ElementAt(itemIndex);
        }
}

public static double Median<T>(this IEnumerable<T> numbers, Func<T, double> selector)
    => (from num in numbers select selector(num)).Median();

In [53]:
public static IEnumerable<Tuple<T, T>> SelectAdjacent<T>(this IEnumerable<T> source)
{
    using (var iterator = source.GetEnumerator())
    {
        if (!iterator.MoveNext())
        {
            yield break;
        }
        T previous = iterator.Current;
        while (iterator.MoveNext())
        {
            yield return new Tuple<T, T>(previous, iterator.Current);
            previous = iterator.Current;
        }
    }
}

In [54]:
private static IEnumerable<T> GetRecords<T, TMap>(string filename) where TMap : ClassMap
{
    using (var reader = new StreamReader(filename))
    {
        using(var csvReader = new CsvReader(reader, CultureInfo.InvariantCulture)) 
        {
            csvReader.Configuration.RegisterClassMap<TMap>();
            return Enumerable.ToList(csvReader.GetRecords<T>());
        }
    }
}

In [55]:
GetRecords<LandRegistryOpenData,LandRegistryOpenDataMap>(filePath).Count()

59727

In [56]:
GetRecords<LandRegistryOpenData,LandRegistryOpenDataMap>(filePath).Take(10)

index,Price,DateOfTransfer,PostCode,PropertyType,Street,City,District,County
0,140000,2006-03-06 00:00:00Z,IG1 1AN,F,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
1,180000,2006-05-26 00:00:00Z,IG1 1AN,F,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
2,140000,2006-03-06 00:00:00Z,IG1 1AN,F,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
3,173250,2005-12-16 00:00:00Z,IG1 1AN,F,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
4,11000000,2017-05-09 00:00:00Z,IG1 1AS,O,THE EXCHANGE,ILFORD,REDBRIDGE,GREATER LONDON
5,10000,2018-09-10 00:00:00Z,IG1 1AS,O,THE EXCHANGE,ILFORD,REDBRIDGE,GREATER LONDON
6,2000,2014-09-05 00:00:00Z,IG1 1BA,O,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON
7,2000,2014-09-05 00:00:00Z,IG1 1BA,O,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON
8,2000,2014-09-05 00:00:00Z,IG1 1BA,O,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON
9,2000,2014-09-05 00:00:00Z,IG1 1BA,O,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON


In [57]:
var dataset  = GetRecords<LandRegistryOpenData,LandRegistryOpenDataMap>(filePath)
                                    .Select(p => new{
                                    Outcode = GetPostcode(p.PostCode),
                                    PropertyType = GetPropertyType(p.PropertyType, propertyTypeLookup),
                                    p.Price,
                                    p.DateOfTransfer,
                                    p.PostCode,
                                    p.Street,
                                    p.City,
                                    p.District,
                                    p.County,
                                });
dataset.Take(10)

index,Outcode,PropertyType,Price,DateOfTransfer,PostCode,Street,City,District,County
0,IG1,Flat,140000,2006-03-06 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
1,IG1,Flat,180000,2006-05-26 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
2,IG1,Flat,140000,2006-03-06 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
3,IG1,Flat,173250,2005-12-16 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
4,IG1,O,11000000,2017-05-09 00:00:00Z,IG1 1AS,THE EXCHANGE,ILFORD,REDBRIDGE,GREATER LONDON
5,IG1,O,10000,2018-09-10 00:00:00Z,IG1 1AS,THE EXCHANGE,ILFORD,REDBRIDGE,GREATER LONDON
6,IG1,O,2000,2014-09-05 00:00:00Z,IG1 1BA,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON
7,IG1,O,2000,2014-09-05 00:00:00Z,IG1 1BA,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON
8,IG1,O,2000,2014-09-05 00:00:00Z,IG1 1BA,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON
9,IG1,O,2000,2014-09-05 00:00:00Z,IG1 1BA,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON


In [58]:
var filterResults = dataset.Where(x => !x.PropertyType.Equals("O"))
                           .Where(x => codes.Contains(x.Outcode));

filterResults.Take(5)

index,Outcode,PropertyType,Price,DateOfTransfer,PostCode,Street,City,District,County
0,IG1,Flat,140000,2006-03-06 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
1,IG1,Flat,180000,2006-05-26 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
2,IG1,Flat,140000,2006-03-06 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
3,IG1,Flat,173250,2005-12-16 00:00:00Z,IG1 1AN,PIPER WAY,ILFORD,REDBRIDGE,GREATER LONDON
4,IG1,Flat,254000,2018-01-09 00:00:00Z,IG1 1BE,CLEMENTS ROAD,ILFORD,REDBRIDGE,GREATER LONDON


### Average and Median Price for all properties

In [59]:
var averageData = filterResults.Where(x => x.DateOfTransfer >= startDate)
                            .GroupBy(t => new { ID = t.Outcode, type = t.PropertyType })
                            .Select(g => new
                            { 
                                NumberOfSales = g.Count(),
                                Average = Math.Round(g.Average(p => p.Price),2),
                                Median  = g.Median(p => p.Price),
                                Outcode = g.Key.ID,
                                PropertyType = g.Key.type,
                               
                            });
averageData

index,NumberOfSales,Average,Median,Outcode,PropertyType
0,232,211340.68,205000,IG1,Flat
1,191,431361.99,410000,IG1,Terraced
2,32,579281.25,585000,IG1,Semi-detached
3,3,960000,1055000,IG1,Detached
4,122,476733.56,477500,IG2,Terraced
5,87,237408.44,230000,IG2,Flat
6,21,542225.71,510000,IG2,Semi-detached
7,114,434545.18,430000,IG3,Terraced
8,38,216263.16,209000,IG3,Flat
9,11,551090.91,530000,IG3,Detached


### Average and Median Price for semi-detached and terraced property

In [60]:
string[] type = {"Semi-detached","Terraced"};
averageData.Where(x => (type.Contains(x.PropertyType)))
            .OrderByDescending(x => x.Median).ThenBy(x => x.PropertyType)

index,NumberOfSales,Average,Median,Outcode,PropertyType
0,32,579281.25,585000,IG1,Semi-detached
1,18,582638.89,571250,IG4,Semi-detached
2,40,535525,542500,IG4,Terraced
3,56,541026.77,527500,IG5,Semi-detached
4,21,542225.71,510000,IG2,Semi-detached
5,29,504410.17,500000,IG3,Semi-detached
6,122,476733.56,477500,IG2,Terraced
7,45,468066.67,470000,IG5,Terraced
8,72,463726.39,455000,IG6,Semi-detached
9,145,444477.45,441000,IG6,Terraced


### Top 10 turnover streets, since 2017


In [61]:
dataset.Where(x => (type.Contains(x.PropertyType))).AsParallel().Take(5);
filterResults.Where(x => x.DateOfTransfer >= startDate)
                    .GroupBy(t => new { ID = t.Outcode, street = t.Street })
                    .OrderByDescending(g => g.Count())
                        .Select(g => new
                        { 
                                NumberOfSales = g.Count(),
                        
                                Outcode = g.Key.ID,
                                Street = g.Key.street,
                               
                            }).Take(10).AsParallel()

index,NumberOfSales,Outcode,Street
0,59,IG1,HIGH ROAD
1,47,IG2,PERTH ROAD
2,46,IG1,RODEN STREET
3,30,IG1,ILFORD HILL
4,14,IG2,EASTERN AVENUE
5,11,IG1,THE DRIVE
6,10,IG1,STAINES ROAD
7,10,IG1,THOROLD ROAD
8,10,IG5,FULLWELL AVENUE
9,10,IG6,TOMSWOOD HILL


In [62]:
var yearMedainResults = filterResults
                        .Where(x => !x.PropertyType.Equals("Flat"))
                        .OrderBy(x => x.DateOfTransfer.Year)
                        .GroupBy(t => new { ID = t.DateOfTransfer.Year })
                                     .Select(g => new
                                     { 
                                        Median  = g.Median(p => p.Price),
                                        Year = g.Key.ID,

                                     });

yearMedainResults.Take(5)

index,Median,Year
0,71500,1995
1,74000,1996
2,82500,1997
3,92000,1998
4,107000,1999


In [63]:
public static IEnumerable<TResult> SelectWithPrevious<TSource, TResult>
    (this IEnumerable<TSource> source,
     Func<TSource, TSource, TResult> projection)
{
    using (var iterator = source.GetEnumerator())
    {
        if (!iterator.MoveNext())
        {
             yield break;
        }
        TSource previous = iterator.Current;
        while (iterator.MoveNext())
        {
            yield return projection(previous, iterator.Current);
            previous = iterator.Current;
        }
    }
}

### House price change in Redbridge

In [64]:
var percentagePriceChange = yearMedainResults.SelectWithPrevious((prev, cur) =>
     new { Year = cur.Year, MedianPrice = cur.Median, PriceChange = Math.Round((cur.Median - prev.Median )/cur.Median*100,2)});

percentagePriceChange

index,Year,MedianPrice,PriceChange
0,1996,74000,3.38
1,1997,82500,10.3
2,1998,92000,10.33
3,1999,107000,14.02
4,2000,130000,17.69
5,2001,149000,12.75
6,2002,184000,19.02
7,2003,220000,16.36
8,2004,242000,9.09
9,2005,245000,1.22
